In [ ]:
# setup RPA environment by installing PHP, Chromium web browser and RPA for Python
!apt-get update; apt install php; apt install chromium-browser; pip install rpa --upgrade

# run r.setup() to download TagUI, modify to run Chromium browser and as root user
import rpa as r; r.setup(); r.dump(r.load('/root/.tagui/src/tagui').replace('"google-chrome"', '"chromium-browser"').replace('$headless_switch', '--no-sandbox'), '/root/.tagui/src/tagui')

# virtual display to run web browser normally (some websites reject headless mode)
!apt install xvfb; pip install pyvirtualdisplay
import pyvirtualdisplay; display = pyvirtualdisplay.Display(); display.start()

In [ ]:
!pip install pandas
!pip install flask-ngrok
!pip install pyngrok

In [ ]:
# testing tagui rpa python package
r.init(visual_automation = True)
r.url('https://google.com')
r.close()

In [ ]:
# Get ngrok token
!ngrok authtoken 22TsxoholZFmuTU6KDK8KZcqT2B_7pxTVG2aCKTQaPum5sQnQ

In [ ]:
# import libraries
from flask_ngrok import run_with_ngrok
from flask import Flask, request, Response
import pandas as pd
from time import sleep
from rpabot_2 import extractinfo
from rpabot_3 import extractpdf

In [ ]:
app = Flask(__name__)
run_with_ngrok(app)

@app.route('/check_query/', methods = ['POST'])
def add_message():
  req_data = request.get_json()
  if 'userid' in req_data:
    userid = req_data['userid']
    print(userid)
  if 'username' in req_data:
    username = req_data['username']
    print(username)
  if 'password' in req_data:
    password = req_data['password']
    print(password)
  return "Done"

@app.route('/rpabot/', methods=['GET', 'POST'])
def rpa():
  req_data = request.get_json()
  if 'userid' in req_data:
    userid = req_data['userid']
  if 'username' in req_data:
    username = req_data['username']
  if 'password' in req_data:
    password = req_data['password']
  df = extractinfo(userid, username, password)
  return Response(df.to_json(orient = 'table'), mimetype='application/json')

@app.route('/rpabot_pdf/', methods=['GET', 'POST'])
def rpa_pdf():
  req_data = request.get_json()
  if 'userid' in req_data:
    userid = req_data['userid']
  if 'username' in req_data:
    username = req_data['username']
  if 'password' in req_data:
    password = req_data['password']
  if 'pdfurl' in req_data:
    pdfurl = req_data['pdfurl']  
  extractpdf(userid, username, password, pdfurl)
  return "Pdf of RFQ downloaded, please check backend"

if __name__ == '__main__':
  app.run()